# Optibook Manual

This notebook contains examples for **all interactions** you can do with optibook (that means inserting orders, getting your positions, etc.). You should use this notebook as a reference/documenation for the system later on when you write more extensive algorithms.

First we do some setup and import the optibook client, which is used to connect to the optibook exchange. If everything is setup correctly, you should see the line "Setup was successful." being printed.

In [24]:
from collections import OrderedDict
def get_outstanding_orders(e,instrument_id):
    orders = e.get_outstanding_orders(instrument_id)
    bids = OrderedDict()
    asks = OrderedDict()
    print(orders)
    for  x in orders:
        if(orders[x].side == "bid"):
            if(bids.get(orders[x].price)):
                bids[orders[x].price].append(orders[x])
            else:
                bids[orders[x].price] = [orders[x]]
        else:
            if(asks.get(orders[x].price)):
                asks[orders[x].price].append(orders[x])
            else:
                asks[orders[x].price] = [orders[x]]
    
    return (bids,asks)

In [41]:
get_outstanding_orders(e,instrument_id)

{312421: <optibook.common_types.OrderStatus object at 0x7f9b20420f98>, 312534: <optibook.common_types.OrderStatus object at 0x7f9b20427390>, 312609: <optibook.common_types.OrderStatus object at 0x7f9b204c9748>, 322082: <optibook.common_types.OrderStatus object at 0x7f9b20420828>, 322892: <optibook.common_types.OrderStatus object at 0x7f9b2154def0>, 322989: <optibook.common_types.OrderStatus object at 0x7f9b201aaf28>, 323715: <optibook.common_types.OrderStatus object at 0x7f9b20427550>, 323731: <optibook.common_types.OrderStatus object at 0x7f9b20427160>, 323842: <optibook.common_types.OrderStatus object at 0x7f9b201aabe0>, 323904: <optibook.common_types.OrderStatus object at 0x7f9b201aab38>}


(OrderedDict([(10.0, [<optibook.common_types.OrderStatus at 0x7f9b20420828>]),
              (11.0,
                <optibook.common_types.OrderStatus at 0x7f9b201aab38>]),
              (12.0,
                <optibook.common_types.OrderStatus at 0x7f9b20427160>])]),
 OrderedDict([(2999.0,
               [<optibook.common_types.OrderStatus at 0x7f9b20420f98>]),
              (2998.0,
               [<optibook.common_types.OrderStatus at 0x7f9b20427390>]),
              (2997.0,
               [<optibook.common_types.OrderStatus at 0x7f9b204c9748>])]))

In [1]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


### Define Your Instrument

An instrument is the term we use for the 'thing' that we are trading. As such, if we are trading BMW stocks, we would call those an instrument. However, BMW options would be a different instrument. 

This field determines which instrument we are trading. By changing it, you will insert trades for a different instrument. You can see all available instruments by looking at the dropdown menu labeled "Instruments" on the visualizer.

In [2]:
instrument_id = 'PHILIPS_A'

### Connect to Exchange

In [3]:
e = Exchange()
a = e.connect()

# you can also define host/user/pass yourself
# when not defined, it is taken from ~/.optibook file if it exists
# if that file does not exists, an error is thrown

# e = Exchange(host='host-to-connect-to')
# a = e.connect(username='your-username', password='your-password')


2022-04-03 03:06:09,377 [asyncio   ] [MainThread  ] Using selector: EpollSelector


## Outstanding Orders, Trades, Current Positions and PnL

In [31]:
# Returns all currently outstanding orders
orders = e.get_outstanding_orders(instrument_id)
for o in orders.values():
    print(o.order_id,o.instrument_id,o.price,o.volume,o.side)

312421 PHILIPS_A 2999.0 1 ask
312534 PHILIPS_A 2998.0 1 ask
312609 PHILIPS_A 2997.0 1 ask


In [6]:
# Returns all trades you have done since the last time this function was called
trades = e.poll_new_trades(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [27]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [22]:
# Returns all current positions
positions = e.get_positions()
for p in positions:
    print(p, positions[p])

PHILIPS_A 39
PHILIPS_B 0


In [8]:
# Returns all current positions with cash invested
positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

PHILIPS_A {'volume': 0, 'cash': 0.0}
PHILIPS_B {'volume': 0, 'cash': 0.0}


In [9]:
# Returns Current PnL based on last Traded Price
pnl = e.get_pnl()
print(pnl)

0.0


## Order Book and Public Trade Ticks

In [10]:
book = e.get_last_price_book(instrument_id)
print(book.timestamp)

2022-04-03 00:44:23.695101


In [12]:
type(book.bids[0])

optibook.common_types.PriceVolume

In [11]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)
for t in tradeticks:
    print(f"[{t.instrument_id}] timesteamp({t.timestamp}) price({t.price}), volume({t.volume}), aggressor_side({t.aggressor_side}), buyer({t.buyer}), seller({t.seller})")

[PHILIPS_A] timesteamp(2022-04-03 00:44:00) price(82.5), volume(5), aggressor_side(bid), buyer(), seller()
[PHILIPS_A] timesteamp(2022-04-03 00:44:04) price(82.10000000000001), volume(15), aggressor_side(ask), buyer(), seller()
[PHILIPS_A] timesteamp(2022-04-03 00:44:04) price(82.60000000000001), volume(15), aggressor_side(bid), buyer(), seller()
[PHILIPS_A] timesteamp(2022-04-03 00:44:04) price(82.60000000000001), volume(5), aggressor_side(bid), buyer(), seller()
[PHILIPS_A] timesteamp(2022-04-03 00:44:04) price(82.60000000000001), volume(15), aggressor_side(bid), buyer(), seller()
[PHILIPS_A] timesteamp(2022-04-03 00:44:04) price(82.60000000000001), volume(15), aggressor_side(bid), buyer(), seller()
[PHILIPS_A] timesteamp(2022-04-03 00:44:05) price(82.60000000000001), volume(15), aggressor_side(bid), buyer(), seller()
[PHILIPS_A] timesteamp(2022-04-03 00:44:14) price(82.7), volume(5), aggressor_side(bid), buyer(), seller()
[PHILIPS_A] timesteamp(2022-04-03 00:44:17) price(82.10000000

In [36]:
# Returns all public tradeticks since the instantiation of the Exchange
tradeticks = e.get_trade_tick_history(instrument_id)
for t in tradeticks:
    print(t)

2022-04-02 20:44:22,963 [client    ] [Thread-5    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [13]:
# See all your outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

## Inserting and Deleting Orders

In [40]:
# Insert bid limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=11, volume=1, side='bid', order_type='limit')
print(f"Order Id: {result}")

Order Id: 323904


In [11]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=2997, volume=1, side='ask', order_type='limit')
print(f"Order Id: {result}")

Order Id: 312609


In [16]:
# Insert bid IOC - This can trade against any resting volume but does not remain in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=445.0, volume=1, side='bid', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 534174


In [17]:
# Insert ask IOC - This can trade against any resting volume but does not remain in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=430.0, volume=1, side='ask', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 534175


In [18]:
# Attempt to delete inserted order by order_id
order_id = 4
result = e.delete_order(instrument_id, order_id=order_id)
print()
print(f"Deleted order id {order_id}: {result}")


Deleted order id 4: False


In [19]:
# Change volume for existing order
order_id = 5
new_volume = 16
result = e.amend_order(instrument_id, order_id=order_id, volume=new_volume)
print(f"Changed volume for order id {order_id} to {new_volume} lots: {result}")

Changed volume for order id 5 to 16 lots: False


In [20]:
# Delete all outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    result = e.delete_order(instrument_id, order_id=o.order_id)
    print(f"Deleted order id {o.order_id}: {result}")

Deleted order id 534165: True


## 'Hack' Out of Positions

In [21]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
print(e.get_positions())
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

{'PHILIPS_A': 22, 'PHILIPS_B': 0}
{'PHILIPS_A': 0, 'PHILIPS_B': 0}
